In [1]:
import pandas as pd
import numpy as np
import requests
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime


In [4]:
# def get_binance_symbols():
#     url = 'https://fapi.binance.com/fapi/v1/ticker/price'
#     r = requests.get(url)
#     data = r.json()
#     return pd.DataFrame(data)



def get_binance_asset(asset, c):
    url = 'https://fapi.binance.com/fapi/v1/klines'
    params = { 'symbol': asset,
                'limit': '720',
               'interval': '1h'
               }
    r = requests.get(url, params)

    data = r.json()
    return pd.DataFrame(data)[c]

# get_binance_asset("BTCUSDT", 4)

# assets = get_binance_symbols()['symbol']  #из-за новых монет тут лимит, возможно, придется уменьшить, потому как возникает ошибка или воспользоваться списком ниже

assets = ['BTCUSDT', 'XTZUSDT', 'LINKUSDT', 'BATUSDT', 'NEOUSDT', 'ZECUSDT', 'DASHUSDT', 'ETHUSDT', 'ETCUSDT', 'LTCUSDT', 'XLMUSDT', 'XRPUSDT', 'ONTUSDT', 'AKROUSDT', 'ZENUSDT', 'DYDXUSDT', 'CTSIUSDT', 'DUSKUSDT',

'BNBUSDT', 'XMRUSDT', 'BCHUSDT', 'EOSUSDT', 'THETAUSDT', 'WAVESUSDT', 'IOSTUSDT', 'ADAUSDT', 'DOTUSDT', 'QTUMUSDT', 'SXPUSDT', 'MKRUSDT', 'CRVUSDT', 'CHZUSDT', 'BTSUSDT', 'TLMUSDT', 'GALAUSDT', 'LPTUSDT',

'MATICUSDT', 'ENJUSDT', 'UNIUSDT', 'MKRUSDT', 'RUNEUSDT', 'BELUSDT', 'BALUSDT', 'REEFUSDT', '1INCHUSDT', 'COMPUSDT', 'SNXUSDT', 'TRBUSDT', 'SRMUSDT', 'LITUSDT', 'UNFIUSDT', 'AUDIOUSDT', 'ARUSDT', 'ENSUSDT',

'NEARUSDT', 'KAVAUSDT', 'STORJUSDT', 'TOMOUSDT', 'AAVEUSDT', 'ANKRUSDT', 'RENUSDT', 'KNCUSDT', 'RLCUSDT', 'ICXUSDT', 'BLZUSDT', 'FTMUSDT', 'HNTUSDT', 'DODOUSDT', 'RVNUSDT', 'RAYUSDT', 'KLAYUSDT', 'PEOPLEUSDT',

'SANDUSDT', 'ATOMUSDT', 'SOLUSDT', 'OGNUSDT', '1000SHIBUSDT', 'BAKEUSDT', 'SKLUSDT', 'FLMUSDT', 'RSRUSDT', 'LRCUSDT', 'CVCUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'SFPUSDT', 'C98USDT', 'ARPAUSDT', 'ANTUSDT',

'DOGEUSDT', 'AVAXUSDT', 'ALGOUSDT', 'ICPUSDT', 'FILUSDT', 'TRXUSDT', 'VETUSDT', 'EGLDUSDT', 'HBARUSDT', 'XEMUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSDT', 'ALICEUSDT', 'ATAUSDT', 'NUUSDT', 'ROSEUSDT',

'IOTAUSDT', 'GRTUSDT', 'KSMUSDT', 'SUSHIUSDT', 'DENTUSDT', 'OCEANUSDT', 'STMXUSDT', 'CELRUSDT', 'HOTUSDT', 'KEEPUSDT', 'GTCUSDT', 'MTLUSDT', 'NKNUSDT', 'SCUSDT', 'DGBUSDT', 'ICPUSDT', 'LINAUSDT', 'ONEUSDT']



crypto_prices = pd.DataFrame()



for a in assets:
    a = a.strip()
    left = len(assets) - assets.index(a)
    print(f'Downloading {a}. {left} left')

    crypto_prices[a] = get_binance_asset(asset = a, c = 4).astype(float)
    crypto_prices['Time'] = get_binance_asset(asset = a, c = 0).astype('datetime64[ms]')



crypto_prices.set_index('Time', inplace=True)
crypto_prices.index = pd.to_datetime(crypto_prices.index, unit='ms')

crypto_prices

crypto_prices.dropna(axis='columns', inplace=True)   # код от Алексея, удаляет колонки с пустыми значениями, фиксит предыдущий мой комментарий


In [ ]:
data_analysis = {'Pairs': [],
                 'P-Value': []}

#data_ratios= {}

for stock1 in crypto_prices.columns:
    for stock2 in crypto_prices.columns:
        if stock1 != stock2:
            if (stock2, stock1) in data_analysis['Pairs']:
                continue

            pairs = stock1, stock2
            ratio = crypto_prices[stock1]/crypto_prices[stock2]
            pvalue = ts.adfuller(ratio)[1]
            # y = crypto_prices[stock1]
            # x = crypto_prices[stock2]
            # model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
            # pvalue = ts.adfuller(model.resid)[1]




            data_analysis['Pairs'].append(pairs)
            data_analysis['P-Value'].append(pvalue)

            #data_ratios[pairs] = ratio

data_analysis = pd.DataFrame(data_analysis).sort_values('P-Value')
data_analysis.set_index('Pairs')



,P-Value
Pairs,


In [ ]:
tradable_pairs_analysis = data_analysis[data_analysis['P-Value'] < 0.02].sort_values('P-Value')

tradable_pairs_analysis.set_index('Pairs')

,P-Value
Pairs,




```
# Выбран кодовый формат
```



In [ ]:
# Get all tradable pairs into a list
stock_pairs = list(tradable_pairs_analysis['Pairs'].tolist())

# Show the Pairs
for pair1, pair2 in stock_pairs:
    print(f"BINANCE:{pair1}/BINANCE:{pair2},")

In [ ]:
# Create a list of unique tradable stocks
list_stock1 = [stock[0] for stock in stock_pairs]
list_stock2 = [stock[1] for stock in stock_pairs]

for stock in list_stock2:
    list_stock1.append(stock)

unique_stock_list = list(set(list_stock1))

# Create a new DataFrame containing all market information for the tradable pairs
tradable_pairs_data = crypto_prices[unique_stock_list]
tradable_pairs_data.index = pd.to_datetime(tradable_pairs_data.index)


In [ ]:
data_ratios= {}
def get_ratios():
    for stock1 in tradable_pairs_data.columns:
        for stock2 in tradable_pairs_data.columns:
            if stock1 != stock2:

                pairs = stock1, stock2
                ratio = tradable_pairs_data[stock1].astype(float)/tradable_pairs_data[stock2].astype(float)
                data_ratios[pairs] = ratio

new_ratios = {}
def get_newratios():
    for stock1 in new_prices.columns:
        for stock2 in new_prices.columns:
            if stock1 != stock2:
                pairs = stock1, stock2
                ratio = new_prices[stock1].astype(float)/new_prices[stock2].astype(float)
                new_ratios[pairs] = ratio



zscores = {}
def get_zscores():
    for pair in data_ratios:
        zscore = (data_ratios[pair] - data_ratios[pair].mean()) / data_ratios[pair].std()
        zscores[pair] = zscore






In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#tradable_pairs_data
get_ratios()
get_zscores()
zscores

{}

In [ ]:
from matplotlib import pyplot as plt


for pair in stock_pairs:
    dev = 3
    plt.figure(figsize=(15,7))
    minwin = 1
    maxwin = 240
    ratios = data_ratios[pair]

    ratios_mavg5 = ratios.rolling(window=minwin, center=False).mean()
    ratios_mavg60 = ratios.rolling(window=maxwin, center=False).mean()
    std_60 = ratios.rolling(window=maxwin, center=False).std()
    zscore_60_5 = (ratios_mavg5 - ratios_mavg60)/std_60

    plt.plot(zscore_60_5.index, zscore_60_5)
    plt.title('Ratio {}'.format(pair))
    plt.axhline(0)
    plt.axhline(dev, linestyle='--')
    plt.axhline(-dev, linestyle='--')
    plt.legend(['Rolling Ratio z-Score', 'Mean', dev, -dev])
    plt.show()

